In [ ]:
import os
import sys

sys.path.append(os.path.join(os.getcwd(), 'Mrlishu'))
if os.path.exists('/content'):
    os.chdir('/content')
    CODE_DIR = 'PHM-Tongji'
    if not os.path.exists(CODE_DIR):
        !git clone https://github.com/MrLishu/PHM-Tongji.git $CODE_DIR
    os.chdir(f'./{CODE_DIR}')

    !git config --global user.email "2601165031@qq.com"
    !git config --global user.name "MrLishu"

    !pip install colabcode
    from colabcode import ColabCode
    ColabCode(port=10000)


In [ ]:
import numpy as np
from preprocess import cwt


raw_data_directory = r'data/raw/PHM2010'
processed_data_directory = r'data/processed/PHM2010'

sampling_rate=50000

resample_number=1024
step=1

totalscale=256
wavename='morl'

c1_resample = np.load(r'data/processed/PHM2010/c1_1024_resample.npy')
c1_label = np.load(r'data/processed/PHM2010/c1_label.npy')
rs_rate = sampling_rate
c1_cwt = cwt(c1_resample, rs_rate, totalscale=totalscale, wavename=wavename, save=False)


In [ ]:
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from sklearn.model_selection import train_test_split

In [ ]:
class MyDataset(Dataset):
    def __init__(self, data, label):
        self.data = data
        self.label = label

    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, index):
        return self.data[index], self.label[index]

In [16]:
x_train, x_test, y_train, y_test = train_test_split(c1_cwt[:, 3:6], c1_label, test_size=0.1, random_state=99)
train_dataset = MyDataset(x_train, y_train)
test_dataset = MyDataset(x_test, y_test)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=4)

In [ ]:
model = models.resnet18(pretrained=False)
# model.conv1 = nn.Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
model.fc = nn.Linear(in_features=512, out_features=1, bias=True)
model

In [ ]:
data, label = next(iter(train_dataloader))
data.shape

In [15]:
model(data)

RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[4, 7, 256, 1024] to have 3 channels, but got 7 channels instead